In [3]:
using Plots
using LinearAlgebra, Symbolics
using ModelingToolkit
using Graphs
using Random: bitrand
using SparseArrays

6.2 Sparse Representations

In the previous chapter, we saw that the computation is getting expensive because of the Q and for large order of Q, our learning rule will become irrelevant. For now, we can enjoy just by saying that CRN can generate and learn different distributions and they can be tuned accordingly as well. But for application, we are limited.
Still we can hope of making our Q easy so that mctt won't become a bad guy here.
We will now see the sparse representation of Q:
We will use the same functions in generation of Q that we used in CHAPTER5VOL2.

In [10]:
# 1. Create bipartite graph between X-nodes (1–n) and H-nodes (n+1-n+m)
function bipartite_graph_visible_hidden(n::Int, m::Int)
    total = n + m
    g = SimpleGraph(total)
    for i in 1:n, j in (n+1):(n+m)
        add_edge!(g, i, j)
    end
    return g
end
# 2. Assign forward/backward rates from parameter matrices a_pph and b_nph
function rates_crn_rule(a_pph::Matrix{T}, a_nph::Matrix{T}, n::Int64, m::Int64) where T <: Real
    g = bipartite_graph_visible_hidden(n, m)
    rates_fwd = Dict{Tuple{Int,Int}, T}()
    rates_bwd = Dict{Tuple{Int,Int}, T}()
    for e in edges(g)
        iX, jH = src(e), dst(e)
        # enforce iX in 1:4, jH in 5:6
        if jH < iX
            iX, jH = jH, iX
        end
        xi = iX                # 1–4
        hj = jH - n            # 1–2
        # Forward rate for Xi^1 + Hj^0 -> Xi^1 + Hj^1
        rates_fwd[(iX, jH)] = a_pph[xi, hj]
        rates_bwd[(iX, jH)] = one(T)
        # Forward rate for Hj^1 + Xi^0 -> Hj^1 + Xi^1
        rates_fwd[(jH, iX)] = a_nph[xi, hj]
        rates_bwd[(jH, iX)] = one(T)
    end
    return rates_fwd, rates_bwd
end

rates_crn_rule (generic function with 1 method)

In [11]:
#Defining Matrix Q using MCTT
function Q_rule(a_pph::Matrix{T}, a_nph::Matrix{T},n,m) where T <: Real
    N = n+m
    S = 2^N
    configs = [reverse(digits(i-1, base=2, pad=N)) for i in 1:S]
    rates_fwd, rates_bwd = rates_crn_rule(a_pph, a_nph,n,m)
    P = spzeros(S, S)
    # (a) Single flips Xi and Hj at rate 1
    for i in 1:S
        state = configs[i]
        for v in 1:N
            new = copy(state)
            new[v] = 1 - new[v]
            j = 1 + foldl((acc,b)->acc*2 + b, new, init=0)
            P[i,j] += one(T)
            P[i,i] -= one(T)
        end
    end
    # (b) Coupled flips along bipartite edges
    g = bipartite_graph_visible_hidden(n, m)
    for i in 1:S
        state = configs[i]
        for e in edges(g)
            iX, jH = src(e), dst(e)
            if jH < iX
                iX, jH = jH, iX
            end
            # if u=Xi and v=Hj
            # forward X→H
            if state[iX]==1 && state[jH]==0
                new = copy(state); new[jH]=1
                j = 1 + foldl((acc,b)->acc*2 + b, new, init=0)
                r = get(rates_fwd,(iX,jH), zero(T))
                P[i,j] += r; P[i,i] -= r
            # backward H→0 on Xi^1+Hj^1→Xi^1+Hj^0
            elseif state[iX]==1 && state[jH]==1
                new = copy(state); new[jH]=0
                j = 1 + foldl((acc,b)->acc*2 + b, new, init=0)
                r = get(rates_bwd,(iX,jH), zero(T))
                P[i,j] += r; P[i,i] -= r
            end
            # now if u=Hj and v=Xi
            # forward H→X
            if state[jH]==1 && state[iX]==0
                new = copy(state); new[iX]=1
                j = 1 + foldl((acc,b)->acc*2 + b, new, init=0)
                r = get(rates_fwd,(jH,iX), zero(T))
                P[i,j] += r; P[i,i] -= r
            # backward X→0 on Hj^1+Xi^1→Hj^1+Xi^0
            elseif state[jH]==1 && state[iX]==1
                new = copy(state); new[iX]=0
                j = 1 + foldl((acc,b)->acc*2 + b, new, init=0)
                r = get(rates_bwd,(jH,iX), zero(T))
                P[i,j] += r; P[i,i] -= r
            end
        end
    end
    return P
end

Q_rule (generic function with 1 method)

Now, first we start with a small example, 
n=3
m=2

In [8]:
a_pph = rand(Float64,3,2)
a_nph = rand(Float64,3,2)
P = Q_rule(a_pph,a_nph,3,2)
P

32×32 SparseMatrixCSC{Float64, Int64} with 192 stored entries:
⎡⢟⣵⠑⢄⠑⢄⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⎤
⎢⠑⢄⢟⣵⠀⠀⠑⢄⠀⠀⠑⢄⠀⠀⠀⠀⎥
⎢⠑⢄⠀⠀⢟⣵⠑⢄⠀⠀⠀⠀⠑⢄⠀⠀⎥
⎢⠀⠀⠑⢄⠑⢄⢟⣵⠀⠀⠀⠀⠀⠀⠑⢄⎥
⎢⠑⢄⠀⠀⠀⠀⠀⠀⢟⣵⠑⢄⠑⢄⠀⠀⎥
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠑⢄⢟⣵⠀⠀⠑⢄⎥
⎢⠀⠀⠀⠀⠑⢄⠀⠀⠑⢄⠀⠀⢟⣵⠑⢄⎥
⎣⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠑⢄⠑⢄⢟⣵⎦

We got some picture which looks symmetric from this view.
Let's have another example of a little higher order.

n=6 m=4

In [9]:
n=6
m=4
a_pph = rand(Float64,n,m)
a_nph = rand(Float64,n,m)
P = Q_rule(a_pph,a_nph,n,m)
P

1024×1024 SparseMatrixCSC{Float64, Int64} with 11264 stored entries:
⎡⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⎥
⎢⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿⎦

Again, we got a similar structure. See 6.2.1

In [12]:
n=8
m=5
a_pph = rand(Float64,n,m)
a_nph = rand(Float64,n,m)
P = Q_rule(a_pph,a_nph,n,m)
P

8192×8192 SparseMatrixCSC{Float64, Int64} with 114688 stored entries:
⎡⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⎥
⎢⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿⎦